In [4]:
import sys
import os
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import  np_utils
import pandas as pd


Using TensorFlow backend.


In [8]:
df = pd.read_csv('fer2013.csv')

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


In [84]:
print(df['Usage'].value_counts())

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64


In [85]:
print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [10]:
X_train,train_y,X_test,test_y = [],[],[],[]

In [11]:
for index, row in df.iterrows():
    val = row['pixels'].split(" ")
    try:
        if 'Training' in row["Usage"]:
            X_train.append(np.array(val,'float32'))
            train_y.append(row["emotion"])
            
        elif 'PublicTest' in row["Usage"]:
                X_test.append(np.array(val,'float32'))
                test_y.append(row['emotion'])
    except:
        print(f"error occured at index:{index} and row:{row}")
            
 

In [29]:
# print(f"X_train sample data:{X_train[0:2]}")
# print(f"train_y sample data:{train_y[0:2]}")
# print(f"X_test sample data:{X_test[0:2]}")
# print(f"test_y sample data:{test_y[0:2]}")

In [30]:
X_train[0]

array([[[-0.6098866 ],
        [-0.4592209 ],
        [-0.40325198],
        ...,
        [-0.7694696 ],
        [-0.90518403],
        [-0.95160526]],

       [[-0.66049284],
        [-0.68162924],
        [-0.694159  ],
        ...,
        [-0.7112763 ],
        [-0.7862608 ],
        [-0.90819967]],

       [[-0.83170736],
        [-0.8952668 ],
        [-0.73603356],
        ...,
        [-0.7924775 ],
        [-0.7261497 ],
        [-0.8613453 ]],

       ...,

       [[-0.33269772],
        [-0.6514918 ],
        [-0.93670547],
        ...,
        [-0.5265123 ],
        [-0.7377333 ],
        [-0.9078971 ]],

       [[-0.5099617 ],
        [-0.43692893],
        [-0.46537215],
        ...,
        [-0.10019822],
        [-0.5580577 ],
        [-0.86989194]],

       [[-0.5095401 ],
        [-0.56317693],
        [-0.3986179 ],
        ...,
        [-0.08629682],
        [-0.0573744 ],
        [-0.41127437]]], dtype=float32)

In [12]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [13]:
# normalising data between 0 and 1

X_train-=np.mean(X_train,axis= 0)
X_train/=np.std(X_train,axis= 0)

X_test-=np.mean(X_test,axis= 0)
X_test/=np.std(X_test,axis= 0)


In [31]:
number_features = 64
number_labels = 7
batch_size = 256
epochs = 50
width,height = 48,48

In [15]:
X_train =X_train.reshape(X_train.shape[0],width,height,1)
X_test =X_test.reshape(X_test.shape[0],width,height,1)

train_y =np_utils.to_categorical(train_y,num_classes=number_labels)
test_y =np_utils.to_categorical(test_y,num_classes=number_labels)


In [16]:
model = Sequential()

In [17]:
# cnn design
# first layer

model.add(Conv2D(number_features,kernel_size=(3,3),
                 activation='relu',input_shape = (X_train.shape[1:])))

model.add(Conv2D(number_features,kernel_size=(3,3),
                 activation='relu'))
          
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Dropout(0.5))

In [18]:
# second layer
          
model.add(Conv2D(number_features,kernel_size=(3,3), activation='relu'))

model.add(Conv2D(number_features,kernel_size=(3,3), activation='relu'))
          
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Dropout(0.5))
          

In [19]:
          
# third layer
          
model.add(Conv2D(2*number_features,kernel_size=(3,3),activation ='relu'))

model.add(Conv2D(2*number_features,kernel_size=(3,3),activation = 'relu'))
          
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

In [20]:
model.add(Flatten())

model.add(Dense(2*2*2*2*number_features, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(2*2*2*2*number_features, activation = 'relu'))
model.add(Dropout(0.2))
          
model.add(Dense(number_labels,activation='softmax'))  

In [21]:
model.compile(loss = categorical_crossentropy,
              optimizer = Adam(), metrics = ['accuracy'])

In [32]:
model.fit(X_train,train_y, batch_size = batch_size,
          epochs= epochs,
          verbose = 1,
          validation_data =(X_test,test_y), shuffle = True)

Train on 28709 samples, validate on 3589 samples
Epoch 1/50
28709/28709 [==============================] - 855s 30ms/step - loss: 1.1875 - accuracy: 0.5472 - val_loss: 1.1999 - val_accuracy: 0.5430
Epoch 2/50
28709/28709 [==============================] - 826s 29ms/step - loss: 1.1578 - accuracy: 0.5567 - val_loss: 1.1863 - val_accuracy: 0.5483
Epoch 3/50
28709/28709 [==============================] - 825s 29ms/step - loss: 1.1181 - accuracy: 0.5730 - val_loss: 1.1713 - val_accuracy: 0.5645
Epoch 4/50
28709/28709 [==============================] - 820s 29ms/step - loss: 1.0925 - accuracy: 0.5812 - val_loss: 1.1561 - val_accuracy: 0.5637
Epoch 5/50
28709/28709 [==============================] - 822s 29ms/step - loss: 1.0601 - accuracy: 0.5953 - val_loss: 1.1388 - val_accuracy: 0.5692
Epoch 6/50
28709/28709 [==============================] - 820s 29ms/step - loss: 1.0368 - accuracy: 0.6040 - val_loss: 1.1575 - val_accuracy: 0.5634
Epoch 7/50
28709/28709 [==============================] -

In [200]:
# fer_json = model.to_json()

In [201]:
# with open("fer.json","w") as json_file:
#     json_file.write(fer_json)

In [202]:
# model.save_weights("fer.h5")

In [33]:
fer_json1 = model.to_json()

with open("fer.json1","w") as json_file:
    json_file.write(fer_json1)
    
model.save_weights("fer_1.h5")